获得经济学人中有notes的刊号和文章名称，用于获取相关音频

In [ ]:
import os
import json
from datetime import datetime

def get_closest_json_by_filename(folder):
    today = datetime.today().date()
    closest_file = None
    min_diff = None

    for file in os.listdir(folder):
        if file.endswith(".json"):
            name = os.path.splitext(file)[0]
            try:
                file_date = datetime.strptime(name, "%Y-%m-%d").date()
                diff = abs((file_date - today).days)
                if min_diff is None or diff < min_diff:
                    min_diff = diff
                    closest_file = file
            except ValueError:
                # 文件名不是日期格式，跳过
                continue

    if closest_file:
        closest_path = os.path.join(folder, closest_file)
        print(f"✅ 最接近今天的文件: {closest_path}")
        with open(closest_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        return data
    else:
        print("❌ 未找到日期格式的 json 文件。")
        return None


folder_path = r"G:\Code\Python\Project\Reader\data\backup\notes"
json_data = get_closest_json_by_filename(folder_path)

pairs = []
seen_articles = set()  # 用 set 去重章节

for entry in json_data:
    bookName = entry.get("bookName", "")
    chapter = entry.get("chapter", "")
    chapterIndex = entry.get("chapterIndex", "")
    if "The Economist" in bookName and chapter not in seen_articles:
        seen_articles.add(chapter)
        pairs.append((bookName, chapter,chapterIndex))

# 按 bookName 排序
pairs.sort(key=lambda x: x[0])

print("\n".join(f"{book} - {chapterIndex} - {chapter}" for book, chapter, chapterIndex in pairs))

# chapterIndex + 6 = real index of audio


In [ ]:
import re
from datetime import datetime
from pathlib import Path
import shutil

def extract_index(filename: Path):
        match = re.match(r"(\d+)", filename.name)
        return int(match.group(1)) if match else 999999

audio_base_dir = Path(r"G:\Book\Economist")
readed_dir = audio_base_dir / "ReadedAudio"

for book, chapter, chapterIndex in pairs:
    print(book)
    # 1. 提取方括号中的日期
    match = re.search(r'\[(.*?)\]', book)
    date_str = match.group(1)  # 'Sep 13th 2025'

    # 2. 清洗并转为标准日期格式
    date_str = re.sub(r'(st|nd|rd|th)', '', date_str)  # 去掉th等后缀
    date_obj = datetime.strptime(date_str, '%b %d %Y')
    formatted_date = date_obj.strftime('%Y-%m-%d')

    # 3. 转换为文件夹格式
    folder_date = date_obj.strftime('%Y%m%d')
    folder_name = f"TEco-{folder_date}音频"
    folder_path = audio_base_dir / folder_name

    mp3_files = [f for f in folder_path.rglob("*.mp3")]
    mp3_files_sorted = sorted(mp3_files, key=extract_index)

    for mp3_file in mp3_files_sorted:
        if extract_index(mp3_file) == chapterIndex + 6:
            dest_file = readed_dir / mp3_file.name
            if dest_file.exists():
                print(f"⚠️ 文件已存在，跳过: {dest_file.name}")
            else:
                shutil.copy2(mp3_file, dest_file)  # 拷贝文件，保留元数据
                print(f"✅ 已拷贝: {mp3_file.name} -> {readed_dir}")
            print(chapter)
            break
    # break  # 示例只处理第一个




In [4]:
"""
更新 Media 牌组中的 definition 和 pos_of_definition
- 重新爬虫获取最新的单词信息
- 更新 Definition 和 POS_Definitions 字段
- 保留其他字段（Examples, Blanked_Examples 等）和查看记录
"""
import sys
from pathlib import Path
import time

# 添加项目路径（code 目录）
# 在 notebook 中，当前目录通常是 code/utils，所以需要回到 code 目录
code_dir = Path.cwd().parent if Path.cwd().name == 'utils' else Path.cwd()
sys.path.insert(0, str(code_dir))

from anki.anki import invoke as anki_invoke, build_html_from_word_info, ensure_pronunciation_audio
from dictionary.dict import get_word_info_by_word
import requests

DECK_NAME = "Media"
SLEEP_TIME = 0.5  # 爬虫间隔，避免请求过快
SKIP_FIRST_N = 0  # 跳过前 n 个笔记（用于从上次停止的地方继续）
ANKI_RETRY_TIMES = 3  # Anki 连接失败时的重试次数
ANKI_RETRY_DELAY = 2  # 每次重试之间的等待时间（秒）

def invoke_with_retry(action: str, retry_times: int = ANKI_RETRY_TIMES, retry_delay: float = ANKI_RETRY_DELAY, **params):
    """
    带重试机制的 Anki invoke 包装函数
    
    Args:
        action: AnkiConnect action 名称
        retry_times: 重试次数
        retry_delay: 每次重试之间的等待时间（秒）
        **params: 传递给 invoke 的参数
    
    Returns:
        AnkiConnect 的响应结果
    
    Raises:
        Exception: 如果所有重试都失败，抛出异常并暂停等待用户处理
    """
    last_error = None
    
    for attempt in range(retry_times):
        try:
            result = anki_invoke(action, **params)
            # 检查是否有错误
            if result and result.get("error"):
                error_msg = result.get("error", "未知错误")
                print(f"  ⚠️  Anki 返回错误: {error_msg}")
                if attempt < retry_times - 1:
                    print(f"  🔄 将在 {retry_delay} 秒后重试 ({attempt + 1}/{retry_times})...")
                    time.sleep(retry_delay)
                    continue
                else:
                    last_error = error_msg
            else:
                return result
        except (requests.RequestException, ConnectionError, TimeoutError) as e:
            last_error = str(e)
            if attempt < retry_times - 1:
                print(f"  ⚠️  Anki 连接失败: {e}")
                print(f"  🔄 将在 {retry_delay} 秒后重试 ({attempt + 1}/{retry_times})...")
                time.sleep(retry_delay)
            else:
                print(f"  ❌ Anki 连接失败: {e}")
        except Exception as e:
            # 其他类型的错误，不重试
            raise e
    
    # 所有重试都失败了，暂停等待用户处理
    print("\n" + "=" * 60)
    print("❌ Anki 连接失败，所有重试都已用尽")
    print(f"   操作: {action}")
    print(f"   错误: {last_error}")
    print("=" * 60)
    print("\n请检查以下事项：")
    print("1. Anki 是否正在运行？")
    print("2. AnkiConnect 插件是否已安装并启用？")
    print("3. 网络连接是否正常？")
    print("\n处理完成后，请按 Enter 键继续...")
    
    # 等待用户输入
    try:
        input()  # 在 notebook 中，这可能需要用户手动继续
        print("✅ 继续处理...")
        # 再试一次
        return anki_invoke(action, **params)
    except KeyboardInterrupt:
        print("\n⚠️  用户中断，停止处理")
        raise
    except Exception as e:
        print(f"\n❌ 继续尝试仍然失败: {e}")
        raise

def update_media_deck_definitions(deck_name: str = DECK_NAME, sleep: float = SLEEP_TIME, skip_first_n: int = SKIP_FIRST_N):
    """
    更新 Media 牌组中所有卡片的 Definition 和 POS_Definitions 字段
    
    Args:
        deck_name: 牌组名称，默认为 "Media"
        sleep: 爬虫请求间隔（秒），避免请求过快
        skip_first_n: 跳过前 n 个笔记，默认为 0（不跳过）
    """
    print(f"开始更新牌组 '{deck_name}' 中的 definition 和 pos_of_definition...")
    if skip_first_n > 0:
        print(f"⚠️  将跳过前 {skip_first_n} 个笔记")
    print("=" * 60)
    
    # 1. 获取牌组中所有笔记
    query = f'deck:"{deck_name}"'
    note_ids = invoke_with_retry("findNotes", query=query).get("result", [])
    
    if not note_ids:
        print(f"❌ 牌组 '{deck_name}' 中没有找到任何笔记")
        return
    
    print(f"✅ 找到 {len(note_ids)} 个笔记")
    if skip_first_n > 0:
        remaining = len(note_ids) - skip_first_n
        print(f"📊 将处理 {remaining} 个笔记（跳过前 {skip_first_n} 个）")
    print("=" * 60)
    
    # 2. 获取所有笔记的详细信息
    notes_info = invoke_with_retry("notesInfo", notes=note_ids).get("result", [])
    
    success_count = 0
    fail_count = 0
    skip_count = 0
    skipped_by_user = 0
    
    # 3. 遍历每个笔记并更新
    for i, note_info in enumerate(notes_info, 1):
        # 跳过前 n 个笔记
        if i <= skip_first_n:
            skipped_by_user += 1
            continue
        note_id = note_info.get("noteId")
        fields = note_info.get("fields", {})
        word_field = fields.get("Word", {})
        word = word_field.get("value", "").strip() if word_field else ""
        
        if not word:
            print(f"[{i}/{len(notes_info)}] ⚠️  跳过：笔记 ID {note_id} 没有 Word 字段")
            skip_count += 1
            continue
        
        print(f"\n[{i}/{len(notes_info)}] 处理单词: {word}")
        
        try:
            # 4. 重新爬虫获取最新的单词信息
            print(f"  正在从 Cambridge Dictionary 获取信息...")
            word_info = get_word_info_by_word(word, sleep=sleep)
            
            if not word_info or not word_info.get("partOfSpeech"):
                print(f"  ⚠️  未获取到单词信息，跳过")
                fail_count += 1
                continue
            
            # 5. 构建新的 Definition 和 POS_Definitions HTML
            generated_fields = build_html_from_word_info(word_info)
            
            # 6. 获取并保留发音音频（如果原笔记有音频，保留；如果没有，尝试添加）
            audio_markup = ensure_pronunciation_audio(word_info)
            
            # 构建更新字段
            update_fields = {}
            
            # 更新 Definition 字段
            new_definition = generated_fields.get("Definition", "")
            if new_definition:
                update_fields["Definition"] = new_definition
                print(f"  ✅ Definition 已更新")
            
            # 更新 POS_Definitions 字段
            new_pos_definitions = generated_fields.get("POS_Definitions", "")
            if new_pos_definitions:
                # 如果有新的音频，添加到 POS_Definitions 开头
                if audio_markup:
                    existing_pos = note_info.get("fields", {}).get("POS_Definitions", {}).get("value", "")
                    # 如果原笔记没有音频标记，添加新的音频
                    if "[sound:" not in existing_pos:
                        new_pos_definitions = f"{audio_markup}\n{new_pos_definitions}"
                update_fields["POS_Definitions"] = new_pos_definitions
                print(f"  ✅ POS_Definitions 已更新")
            
            # 7. 更新笔记（只更新 Definition 和 POS_Definitions，保留其他字段）
            if update_fields:
                try:
                    result = invoke_with_retry("updateNoteFields", note={"id": note_id, "fields": update_fields})
                    
                    if result and not result.get("error"):
                        print(f"  ✅ 笔记更新成功")
                        success_count += 1
                    else:
                        error_msg = result.get("error", "未知错误") if result else "无响应"
                        print(f"  ❌ 更新失败: {error_msg}")
                        fail_count += 1
                except Exception as e:
                    print(f"  ❌ 更新失败（连接问题）: {e}")
                    fail_count += 1
            else:
                print(f"  ⚠️  没有可更新的字段")
                skip_count += 1
                
        except Exception as e:
            print(f"  ❌ 处理失败: {e}")
            import traceback
            traceback.print_exc()
            fail_count += 1
    
    # 8. 输出统计信息
    print("\n" + "=" * 60)
    print("更新完成！")
    print(f"成功: {success_count} 个")
    print(f"失败: {fail_count} 个")
    print(f"跳过: {skip_count} 个")
    if skipped_by_user > 0:
        print(f"用户跳过: {skipped_by_user} 个（前 {skip_first_n} 个）")
    print(f"总计: {len(notes_info)} 个")
    print("=" * 60)

# 执行更新
# 修改 SKIP_FIRST_N 的值来跳过前 n 个笔记（例如：SKIP_FIRST_N = 50 表示跳过前 50 个）
if __name__ == "__main__" or True:  # 在 notebook 中总是执行
    update_media_deck_definitions(deck_name=DECK_NAME, sleep=SLEEP_TIME, skip_first_n=SKIP_FIRST_N)


开始更新牌组 'Media' 中的 definition 和 pos_of_definition...
✅ 找到 399 个笔记

[1/399] 处理单词: twilight
  正在从 Cambridge Dictionary 获取信息...
  ✅ Definition 已更新
  ✅ POS_Definitions 已更新
  ✅ 笔记更新成功

[2/399] 处理单词: encapsulation
  正在从 Cambridge Dictionary 获取信息...
  ✅ Definition 已更新
  ✅ POS_Definitions 已更新
  ✅ 笔记更新成功

[3/399] 处理单词: transcend
  正在从 Cambridge Dictionary 获取信息...
  ✅ Definition 已更新
  ✅ POS_Definitions 已更新
  ✅ 笔记更新成功

[4/399] 处理单词: vest
  正在从 Cambridge Dictionary 获取信息...
  ✅ Definition 已更新
  ✅ POS_Definitions 已更新
  ✅ 笔记更新成功

[5/399] 处理单词: clipboard
  正在从 Cambridge Dictionary 获取信息...
  ✅ Definition 已更新
  ✅ POS_Definitions 已更新
  ✅ 笔记更新成功

[6/399] 处理单词: holocaust
  正在从 Cambridge Dictionary 获取信息...
  ✅ Definition 已更新
  ✅ POS_Definitions 已更新
  ✅ 笔记更新成功

[7/399] 处理单词: fission
  正在从 Cambridge Dictionary 获取信息...
  ✅ Definition 已更新
  ✅ POS_Definitions 已更新
  ✅ 笔记更新成功

[8/399] 处理单词: Armageddon
  正在从 Cambridge Dictionary 获取信息...
  ✅ Definition 已更新
  ✅ POS_Definitions 已更新
  ✅ 笔记更新成功

[9/399] 处理单词: remnant
  正

In [7]:
"""
检查并上传 Media 牌组中缺失的 mp3 文件
- 从 Examples 字段中提取 mp3 文件名
- 检查这些文件是否存在于 Anki 媒体库
- 如果不存在，从原始音频目录中找到对应的 mp3 文件并上传
"""
import sys
from pathlib import Path
import re
import base64
import time

# 添加项目路径（code 目录）
code_dir = Path.cwd().parent if Path.cwd().name == 'utils' else Path.cwd()
sys.path.insert(0, str(code_dir))

from anki.anki import invoke as anki_invoke
from movie.import_to_anki import store_media_file, find_media_files

DECK_NAME = "Media"
SKIP_FIRST_N = 0  # 跳过前 n 个笔记
# 配置音频目录（根据实际情况修改）
# 可以配置多个可能的音频目录
# 在 notebook 中，需要根据实际项目路径调整
base_dir = Path.cwd().parent.parent if Path.cwd().name == 'utils' else Path.cwd().parent.parent
AUDIO_DIRS = [
    base_dir / 'data' / 'source' / 'Tenet' / 'audio',
    base_dir / 'data' / 'source' / 'Interstellar' / 'audio',
    base_dir / 'data' / 'source' / 'The Silence of the Lambs' / 'audio',
    base_dir / 'data' / 'source' / 'Green Book' / 'audio',
]
# 过滤掉不存在的目录
AUDIO_DIRS = [d for d in AUDIO_DIRS if d.exists()]

def invoke_with_retry(action: str, retry_times: int = 3, retry_delay: float = 2, **params):
    """带重试机制的 Anki invoke 包装函数"""
    last_error = None
    
    for attempt in range(retry_times):
        try:
            result = anki_invoke(action, **params)
            if result and result.get("error"):
                error_msg = result.get("error", "未知错误")
                if attempt < retry_times - 1:
                    print(f"  ⚠️  Anki 返回错误: {error_msg}, 将在 {retry_delay} 秒后重试...")
                    time.sleep(retry_delay)
                    continue
                else:
                    last_error = error_msg
            else:
                return result
        except Exception as e:
            last_error = str(e)
            if attempt < retry_times - 1:
                print(f"  ⚠️  Anki 连接失败: {e}, 将在 {retry_delay} 秒后重试...")
                time.sleep(retry_delay)
            else:
                raise
    
    if last_error:
        raise Exception(f"Anki 操作失败: {last_error}")
    return result

def extract_mp3_filenames_from_examples(examples_html: str) -> list:
    """从 Examples 字段的 HTML 中提取所有 mp3 文件名"""
    mp3_files = []
    
    # 匹配 audioEl.src = 'filename.mp3' 格式
    pattern1 = r"audioEl\.src\s*=\s*['\"]([^'\"]+\.mp3)['\"]"
    matches1 = re.findall(pattern1, examples_html, re.IGNORECASE)
    mp3_files.extend(matches1)
    
    # 匹配 [sound:filename.mp3] 格式
    pattern2 = r"\[sound:([^\]]+\.mp3)\]"
    matches2 = re.findall(pattern2, examples_html, re.IGNORECASE)
    mp3_files.extend(matches2)
    
    # 去重并返回
    return list(set(mp3_files))

def check_media_file_exists(filename: str) -> bool:
    """
    检查 Anki 媒体库中是否存在指定文件
    注意：AnkiConnect 没有直接检查文件是否存在的方法
    这里我们总是返回 False，让上传逻辑处理（如果文件已存在，上传会成功但不会覆盖）
    """
    # AnkiConnect 的 storeMediaFile 如果文件已存在，会返回成功但不会覆盖
    # 所以我们直接返回 False，让上传逻辑处理
    return False

def convert_filename_to_new_format(old_filename: str) -> str:
    """
    将旧格式文件名转换为新格式
    旧格式：单词_序号.mp3 (如 transcend_03.mp3)
    新格式：序号_单词.mp3 (如 03_transcend.mp3)
    """
    name_without_ext = Path(old_filename).stem
    ext = Path(old_filename).suffix
    
    # 尝试匹配旧格式：单词_序号
    match = re.match(r'^(.+?)_(\d+)$', name_without_ext)
    if match:
        word = match.group(1)
        number = match.group(2)
        # 转换为新格式：序号_单词
        new_filename = f"{number}_{word}{ext}"
        return new_filename
    
    # 如果已经是新格式或无法匹配，返回原文件名
    return old_filename

def find_mp3_file_in_dirs(filename: str, audio_dirs: list) -> tuple:
    """
    在音频目录中查找对应的 mp3 文件
    返回: (文件路径, 新格式文件名) 或 (None, None)
    如果找到旧格式文件，会转换为新格式文件名
    """
    filename_only = Path(filename).name
    name_without_ext = Path(filename_only).stem
    
    # 尝试匹配旧格式：单词_序号
    match_old = re.match(r'^(.+?)_(\d+)$', name_without_ext)
    # 尝试匹配新格式：序号_单词
    match_new = re.match(r'^(\d+)_(.+)$', name_without_ext)
    
    for audio_dir in audio_dirs:
        if not audio_dir or not audio_dir.exists():
            continue
        
        # 1. 直接查找文件名（新格式或旧格式）
        mp3_file = audio_dir / filename_only
        if mp3_file.exists():
            # 检查是否是旧格式，如果是则转换为新格式
            new_filename = convert_filename_to_new_format(filename_only)
            return mp3_file, new_filename
        
        # 2. 如果文件名是旧格式（单词_序号），尝试查找新格式文件
        if match_old:
            word = match_old.group(1)
            number = match_old.group(2)
            # 查找新格式：序号_单词.mp3
            new_pattern = f"{number}_{word}.mp3"
            new_file = audio_dir / new_pattern
            if new_file.exists():
                return new_file, new_pattern
        
        # 3. 如果文件名是新格式（序号_单词），直接查找
        if match_new:
            number = match_new.group(1)
            word = match_new.group(2)
            pattern = f"{number}_{word}.mp3"
            new_file = audio_dir / pattern
            if new_file.exists():
                return new_file, pattern
        
        # 4. 如果文件名是旧格式，尝试通过单词查找旧格式文件
        if match_old:
            word = match_old.group(1)
            pattern_old = f"{word}_*.mp3"
            matches = list(audio_dir.glob(pattern_old))
            if matches:
                # 找到旧格式文件，转换为新格式
                old_file = matches[0]
                new_filename = convert_filename_to_new_format(old_file.name)
                return old_file, new_filename
    
    return None, None

def check_and_upload_missing_mp3s(deck_name: str = DECK_NAME, skip_first_n: int = SKIP_FIRST_N, audio_dirs: list = None):
    """
    检查 Media 牌组中缺失的 mp3 文件并上传
    
    Args:
        deck_name: 牌组名称
        skip_first_n: 跳过前 n 个笔记
        audio_dirs: 音频目录列表，如果为 None 则使用默认配置
    """
    if audio_dirs is None:
        audio_dirs = AUDIO_DIRS
    
    print(f"开始检查牌组 '{deck_name}' 中缺失的 mp3 文件...")
    if skip_first_n > 0:
        print(f"⚠️  将跳过前 {skip_first_n} 个笔记")
    print("=" * 60)
    
    # 显示配置的音频目录
    print("配置的音频目录:")
    for i, audio_dir in enumerate(audio_dirs, 1):
        exists = "✅" if audio_dir and audio_dir.exists() else "❌"
        print(f"  {i}. {exists} {audio_dir}")
    print("=" * 60)
    
    # 1. 获取牌组中所有笔记
    query = f'deck:"{deck_name}"'
    note_ids = invoke_with_retry("findNotes", query=query).get("result", [])
    
    if not note_ids:
        print(f"❌ 牌组 '{deck_name}' 中没有找到任何笔记")
        return
    
    print(f"✅ 找到 {len(note_ids)} 个笔记")
    if skip_first_n > 0:
        remaining = len(note_ids) - skip_first_n
        print(f"📊 将处理 {remaining} 个笔记（跳过前 {skip_first_n} 个）")
    print("=" * 60)
    
    # 2. 获取所有笔记的详细信息
    notes_info = invoke_with_retry("notesInfo", notes=note_ids).get("result", [])
    
    success_count = 0
    fail_count = 0
    skip_count = 0
    skipped_by_user = 0
    
    # 3. 遍历每个笔记
    for i, note_info in enumerate(notes_info, 1):
        # 跳过前 n 个笔记
        if i <= skip_first_n:
            skipped_by_user += 1
            continue
        
        note_id = note_info.get("noteId")
        fields = note_info.get("fields", {})
        word_field = fields.get("Word", {})
        word = word_field.get("value", "").strip() if word_field else ""
        examples_field = fields.get("Examples", {})
        examples_html = examples_field.get("value", "") if examples_field else ""
        
        if not word:
            print(f"\n[{i}/{len(notes_info)}] ⚠️  跳过：笔记 ID {note_id} 没有 Word 字段")
            skip_count += 1
            continue
        
        if not examples_html:
            print(f"\n[{i}/{len(notes_info)}] ⚠️  跳过：单词 '{word}' 没有 Examples 字段")
            skip_count += 1
            continue
        
        print(f"\n[{i}/{len(notes_info)}] 处理单词: {word}")
        
        try:
            # 4. 从 Examples 字段中提取 mp3 文件名
            mp3_filenames = extract_mp3_filenames_from_examples(examples_html)
            
            if not mp3_filenames:
                print(f"  ⚠️  未找到 mp3 文件引用")
                skip_count += 1
                continue
            
            print(f"  📋 找到 {len(mp3_filenames)} 个 mp3 文件引用: {', '.join(mp3_filenames)}")
            
            # 5. 处理每个 mp3 文件（检查并上传）
            for mp3_filename in mp3_filenames:
                # 在音频目录中查找文件
                mp3_file, new_filename = find_mp3_file_in_dirs(mp3_filename, audio_dirs)
                
                if not mp3_file:
                    print(f"  ⚠️  未找到源文件: {mp3_filename}")
                    fail_count += 1
                    continue
                
                # 如果文件名需要转换（旧格式转新格式）
                if new_filename != mp3_filename:
                    print(f"  🔄 文件名格式转换: {mp3_filename} -> {new_filename}")
                    upload_filename = new_filename
                else:
                    upload_filename = mp3_filename
                
                print(f"  📁 找到源文件: {mp3_file.name}")
                
                # 上传文件（使用新格式文件名）
                try:
                    if store_media_file(str(mp3_file), upload_filename):
                        print(f"  ✅ 成功上传: {upload_filename}")
                        success_count += 1
                        
                        # 如果文件名被转换了，需要更新 Examples 字段中的引用
                        if new_filename != mp3_filename:
                            print(f"  🔄 需要更新 Examples 字段中的文件名引用")
                            # 更新 Examples 字段中的文件名
                            updated_examples = examples_html.replace(mp3_filename, upload_filename)
                            if updated_examples != examples_html:
                                try:
                                    result = invoke_with_retry("updateNoteFields", 
                                                             note={"id": note_id, 
                                                                   "fields": {"Examples": updated_examples}})
                                    if result and not result.get("error"):
                                        print(f"  ✅ 已更新 Examples 字段中的文件名引用")
                                    else:
                                        print(f"  ⚠️  更新 Examples 字段失败，但文件已上传")
                                except Exception as e:
                                    print(f"  ⚠️  更新 Examples 字段异常: {e}")
                    else:
                        print(f"  ❌ 上传失败: {upload_filename}")
                        fail_count += 1
                except Exception as e:
                    print(f"  ❌ 上传异常: {upload_filename} - {e}")
                    fail_count += 1
                    
        except Exception as e:
            print(f"  ❌ 处理失败: {e}")
            import traceback
            traceback.print_exc()
            fail_count += 1
    
    # 7. 输出统计信息
    print("\n" + "=" * 60)
    print("检查完成！")
    print(f"成功上传: {success_count} 个")
    print(f"失败: {fail_count} 个")
    print(f"跳过: {skip_count} 个")
    if skipped_by_user > 0:
        print(f"用户跳过: {skipped_by_user} 个（前 {skip_first_n} 个）")
    print(f"总计: {len(notes_info)} 个")
    print("=" * 60)

# 执行检查
# 修改 SKIP_FIRST_N 的值来跳过前 n 个笔记
# 修改 AUDIO_DIRS 来配置音频目录
if __name__ == "__main__" or True:  # 在 notebook 中总是执行
    check_and_upload_missing_mp3s(deck_name=DECK_NAME, skip_first_n=SKIP_FIRST_N, audio_dirs=AUDIO_DIRS)


开始检查牌组 'Media' 中缺失的 mp3 文件...
配置的音频目录:
  1. ✅ /Users/xrn/Documents/Code/Python/Reader/data/source/Tenet/audio
  2. ✅ /Users/xrn/Documents/Code/Python/Reader/data/source/Interstellar/audio
  3. ✅ /Users/xrn/Documents/Code/Python/Reader/data/source/The Silence of the Lambs/audio
✅ 找到 399 个笔记

[1/399] 处理单词: twilight
  📋 找到 1 个 mp3 文件引用: 01_twilight.mp3
  📁 找到源文件: 01_twilight.mp3
  ✅ 成功上传: 01_twilight.mp3

[2/399] 处理单词: encapsulation
  📋 找到 1 个 mp3 文件引用: 02_encapsulation.mp3
  📁 找到源文件: 02_encapsulation.mp3
  ✅ 成功上传: 02_encapsulation.mp3

[3/399] 处理单词: transcend
  📋 找到 1 个 mp3 文件引用: 03_transcend.mp3
  📁 找到源文件: 03_transcend.mp3
  ✅ 成功上传: 03_transcend.mp3

[4/399] 处理单词: vest
  📋 找到 1 个 mp3 文件引用: 05_vest.mp3
  📁 找到源文件: 05_vest.mp3
  ✅ 成功上传: 05_vest.mp3

[5/399] 处理单词: clipboard
  📋 找到 1 个 mp3 文件引用: 06_clipboard.mp3
  📁 找到源文件: 06_clipboard.mp3
  ✅ 成功上传: 06_clipboard.mp3

[6/399] 处理单词: holocaust
  📋 找到 1 个 mp3 文件引用: 07_holocaust.mp3
  📁 找到源文件: 07_holocaust.mp3
  ✅ 成功上传: 07_holocaust.mp3

[7/399] 

In [9]:
"""
处理所有 tag 为 "tenet" 的卡牌，将 mp3 文件与 reference.mp3 一致后重新上传
- 查找所有 tag 为 "tenet" 的笔记
- 从 Examples 字段中提取 mp3 文件名
- 找到对应的源文件
- 使用 reference.mp3 的 LUFS 值标准化音频音量
- 重新上传处理后的文件
"""
import sys
from pathlib import Path
import re
import time
import tempfile
import os

# 添加项目路径（code 目录）
code_dir = Path.cwd().parent if Path.cwd().name == 'utils' else Path.cwd()
sys.path.insert(0, str(code_dir))

from anki.anki import invoke as anki_invoke
from movie.import_to_anki import store_media_file
from movie.extract_audio import get_audio_lufs, normalize_audio_volume

DECK_NAME = "Media"
TAG_NAME = "tenet"
SKIP_FIRST_N = 0  # 跳过前 n 个笔记

# 配置路径
base_dir = Path.cwd().parent.parent if Path.cwd().name == 'utils' else Path.cwd().parent.parent
REFERENCE_AUDIO = base_dir / 'data' / 'source' / 'reference.mp3'
AUDIO_DIRS = [
    base_dir / 'data' / 'source' / 'Tenet' / 'audio',
    base_dir / 'data' / 'source' / 'Interstellar' / 'audio',
]
# 过滤掉不存在的目录
AUDIO_DIRS = [d for d in AUDIO_DIRS if d.exists()]

def invoke_with_retry(action: str, retry_times: int = 3, retry_delay: float = 2, **params):
    """带重试机制的 Anki invoke 包装函数"""
    last_error = None
    
    for attempt in range(retry_times):
        try:
            result = anki_invoke(action, **params)
            if result and result.get("error"):
                error_msg = result.get("error", "未知错误")
                if attempt < retry_times - 1:
                    print(f"  ⚠️  Anki 返回错误: {error_msg}, 将在 {retry_delay} 秒后重试...")
                    time.sleep(retry_delay)
                    continue
                else:
                    last_error = error_msg
            else:
                return result
        except Exception as e:
            last_error = str(e)
            if attempt < retry_times - 1:
                print(f"  ⚠️  Anki 连接失败: {e}, 将在 {retry_delay} 秒后重试...")
                time.sleep(retry_delay)
            else:
                raise
    
    if last_error:
        raise Exception(f"Anki 操作失败: {last_error}")
    return result

def extract_mp3_filenames_from_examples(examples_html: str) -> list:
    """从 Examples 字段的 HTML 中提取所有 mp3 文件名"""
    mp3_files = []
    
    # 匹配 audioEl.src = 'filename.mp3' 格式
    pattern1 = r"audioEl\.src\s*=\s*['\"]([^'\"]+\.mp3)['\"]"
    matches1 = re.findall(pattern1, examples_html, re.IGNORECASE)
    mp3_files.extend(matches1)
    
    # 匹配 [sound:filename.mp3] 格式
    pattern2 = r"\[sound:([^\]]+\.mp3)\]"
    matches2 = re.findall(pattern2, examples_html, re.IGNORECASE)
    mp3_files.extend(matches2)
    
    # 去重并返回
    return list(set(mp3_files))

def find_mp3_file_in_dirs(filename: str, audio_dirs: list) -> Path:
    """在音频目录中查找对应的 mp3 文件"""
    filename_only = Path(filename).name
    
    for audio_dir in audio_dirs:
        if not audio_dir or not audio_dir.exists():
            continue
        
        # 直接查找文件名
        mp3_file = audio_dir / filename_only
        if mp3_file.exists():
            return mp3_file
        
        # 尝试匹配数字_单词格式（新格式）
        name_without_ext = Path(filename_only).stem
        match_new = re.match(r'^(\d+)_(.+)$', name_without_ext)
        if match_new:
            number = match_new.group(1)
            word = match_new.group(2)
            pattern = f"{number}_{word}.mp3"
            new_file = audio_dir / pattern
            if new_file.exists():
                return new_file
        
        # 尝试匹配单词_数字格式（旧格式）
        match_old = re.match(r'^(.+?)_(\d+)$', name_without_ext)
        if match_old:
            word = match_old.group(1)
            number = match_old.group(2)
            # 查找新格式
            new_pattern = f"{number}_{word}.mp3"
            new_file = audio_dir / new_pattern
            if new_file.exists():
                return new_file
            # 查找旧格式
            old_pattern = f"{word}_{number}.mp3"
            old_file = audio_dir / old_pattern
            if old_file.exists():
                return old_file
    
    return None

def normalize_and_upload_tenet_mp3s(deck_name: str = DECK_NAME, tag_name: str = TAG_NAME, 
                                    skip_first_n: int = SKIP_FIRST_N, 
                                    reference_audio: Path = REFERENCE_AUDIO,
                                    audio_dirs: list = None):
    """
    处理所有 tag 为指定标签的卡牌，将 mp3 文件与 reference.mp3 一致后重新上传
    
    Args:
        deck_name: 牌组名称
        tag_name: 标签名称
        skip_first_n: 跳过前 n 个笔记
        reference_audio: 参考音频文件路径
        audio_dirs: 音频目录列表
    """
    if audio_dirs is None:
        audio_dirs = AUDIO_DIRS
    
    print(f"开始处理牌组 '{deck_name}' 中 tag 为 '{tag_name}' 的卡牌...")
    if skip_first_n > 0:
        print(f"⚠️  将跳过前 {skip_first_n} 个笔记")
    print("=" * 60)
    
    # 检查参考音频文件
    if not reference_audio.exists():
        print(f"❌ 参考音频文件不存在: {reference_audio}")
        return
    
    print(f"📁 参考音频: {reference_audio}")
    
    # 获取参考音频的 LUFS 值
    print("正在分析参考音频的 LUFS 值...")
    target_lufs = get_audio_lufs(str(reference_audio))
    if target_lufs is None:
        print("⚠️  无法获取参考音频 LUFS 值，使用默认值 -23.0")
        target_lufs = -23.0
    else:
        print(f"✅ 参考音频 LUFS 值: {target_lufs:.2f}")
    
    print("=" * 60)
    
    # 显示配置的音频目录
    print("配置的音频目录:")
    for i, audio_dir in enumerate(audio_dirs, 1):
        exists = "✅" if audio_dir and audio_dir.exists() else "❌"
        print(f"  {i}. {exists} {audio_dir}")
    print("=" * 60)
    
    # 1. 查找所有 tag 为指定标签的笔记
    query = f'deck:"{deck_name}" tag:{tag_name}'
    note_ids = invoke_with_retry("findNotes", query=query).get("result", [])
    
    if not note_ids:
        print(f"❌ 牌组 '{deck_name}' 中没有找到 tag 为 '{tag_name}' 的笔记")
        return
    
    print(f"✅ 找到 {len(note_ids)} 个笔记（tag: {tag_name}）")
    if skip_first_n > 0:
        remaining = len(note_ids) - skip_first_n
        print(f"📊 将处理 {remaining} 个笔记（跳过前 {skip_first_n} 个）")
    print("=" * 60)
    
    # 2. 获取所有笔记的详细信息
    notes_info = invoke_with_retry("notesInfo", notes=note_ids).get("result", [])
    
    success_count = 0
    fail_count = 0
    skip_count = 0
    skipped_by_user = 0
    
    # 3. 遍历每个笔记
    for i, note_info in enumerate(notes_info, 1):
        # 跳过前 n 个笔记
        if i <= skip_first_n:
            skipped_by_user += 1
            continue
        
        note_id = note_info.get("noteId")
        fields = note_info.get("fields", {})
        word_field = fields.get("Word", {})
        word = word_field.get("value", "").strip() if word_field else ""
        examples_field = fields.get("Examples", {})
        examples_html = examples_field.get("value", "") if examples_field else ""
        
        if not word:
            print(f"\n[{i}/{len(notes_info)}] ⚠️  跳过：笔记 ID {note_id} 没有 Word 字段")
            skip_count += 1
            continue
        
        if not examples_html:
            print(f"\n[{i}/{len(notes_info)}] ⚠️  跳过：单词 '{word}' 没有 Examples 字段")
            skip_count += 1
            continue
        
        print(f"\n[{i}/{len(notes_info)}] 处理单词: {word}")
        
        try:
            # 4. 从 Examples 字段中提取 mp3 文件名
            mp3_filenames = extract_mp3_filenames_from_examples(examples_html)
            
            if not mp3_filenames:
                print(f"  ⚠️  未找到 mp3 文件引用")
                skip_count += 1
                continue
            
            print(f"  📋 找到 {len(mp3_filenames)} 个 mp3 文件引用: {', '.join(mp3_filenames)}")
            
            # 5. 处理每个 mp3 文件
            for mp3_filename in mp3_filenames:
                # 在音频目录中查找源文件
                source_file = find_mp3_file_in_dirs(mp3_filename, audio_dirs)
                
                if not source_file:
                    print(f"  ⚠️  未找到源文件: {mp3_filename}")
                    fail_count += 1
                    continue
                
                print(f"  📁 找到源文件: {source_file.name}")
                
                # 6. 标准化音频音量
                print(f"  🔄 正在标准化音频音量（目标 LUFS: {target_lufs:.2f}）...")
                
                # 创建临时文件
                with tempfile.NamedTemporaryFile(suffix='.mp3', delete=False) as tmp_file:
                    temp_output = tmp_file.name
                
                try:
                    # 标准化音量
                    if normalize_audio_volume(str(source_file), temp_output, target_lufs):
                        print(f"  ✅ 音频标准化成功")
                        
                        # 7. 上传处理后的文件
                        if store_media_file(temp_output, mp3_filename):
                            print(f"  ✅ 成功上传: {mp3_filename}")
                            success_count += 1
                        else:
                            print(f"  ❌ 上传失败: {mp3_filename}")
                            fail_count += 1
                    else:
                        print(f"  ❌ 音频标准化失败: {mp3_filename}")
                        fail_count += 1
                except Exception as e:
                    print(f"  ❌ 处理异常: {mp3_filename} - {e}")
                    fail_count += 1
                finally:
                    # 清理临时文件
                    try:
                        if os.path.exists(temp_output):
                            os.remove(temp_output)
                    except:
                        pass
                    
        except Exception as e:
            print(f"  ❌ 处理失败: {e}")
            import traceback
            traceback.print_exc()
            fail_count += 1
    
    # 8. 输出统计信息
    print("\n" + "=" * 60)
    print("处理完成！")
    print(f"成功处理并上传: {success_count} 个")
    print(f"失败: {fail_count} 个")
    print(f"跳过: {skip_count} 个")
    if skipped_by_user > 0:
        print(f"用户跳过: {skipped_by_user} 个（前 {skip_first_n} 个）")
    print(f"总计: {len(notes_info)} 个")
    print("=" * 60)

# 执行处理
# 修改 SKIP_FIRST_N 的值来跳过前 n 个笔记
# 修改 TAG_NAME 来指定不同的标签
if __name__ == "__main__" or True:  # 在 notebook 中总是执行
    normalize_and_upload_tenet_mp3s(deck_name=DECK_NAME, tag_name=TAG_NAME, 
                                    skip_first_n=SKIP_FIRST_N, 
                                    reference_audio=REFERENCE_AUDIO,
                                    audio_dirs=AUDIO_DIRS)


开始处理牌组 'Media' 中 tag 为 'tenet' 的卡牌...
📁 参考音频: /Users/xrn/Documents/Code/Python/Reader/data/source/reference.mp3
正在分析参考音频的 LUFS 值...
✅ 参考音频 LUFS 值: -12.17
配置的音频目录:
  1. ✅ /Users/xrn/Documents/Code/Python/Reader/data/source/Tenet/audio
  2. ✅ /Users/xrn/Documents/Code/Python/Reader/data/source/Interstellar/audio
✅ 找到 123 个笔记（tag: tenet）

[1/123] 处理单词: twilight
  📋 找到 1 个 mp3 文件引用: 01_twilight.mp3
  📁 找到源文件: 01_twilight.mp3
  🔄 正在标准化音频音量（目标 LUFS: -12.17）...
  ✅ 音频标准化成功
  ✅ 成功上传: 01_twilight.mp3

[2/123] 处理单词: encapsulation
  📋 找到 1 个 mp3 文件引用: 02_encapsulation.mp3
  📁 找到源文件: 02_encapsulation.mp3
  🔄 正在标准化音频音量（目标 LUFS: -12.17）...
  ✅ 音频标准化成功
  ✅ 成功上传: 02_encapsulation.mp3

[3/123] 处理单词: transcend
  📋 找到 1 个 mp3 文件引用: 03_transcend.mp3
  📁 找到源文件: 03_transcend.mp3
  🔄 正在标准化音频音量（目标 LUFS: -12.17）...
  ✅ 音频标准化成功
  ✅ 成功上传: 03_transcend.mp3

[4/123] 处理单词: vest
  📋 找到 1 个 mp3 文件引用: 05_vest.mp3
  📁 找到源文件: 05_vest.mp3
  🔄 正在标准化音频音量（目标 LUFS: -12.17）...
  ✅ 音频标准化成功
  ✅ 成功上传: 05_vest.mp3

[5/123] 处理单词: 

In [ ]:
"""
为 Media 牌组中缺少读音的卡片重新爬虫并上传 mp3 发音
- 查找所有 Media 牌组中的笔记
- 检查 Pronunciation 和 POS_Definitions 字段是否包含 [sound:...] 标记
- 如果没有，重新从 Cambridge Dictionary 爬虫获取单词信息
- 下载发音音频并上传到 Anki
- 更新 Pronunciation 和 POS_Definitions 字段
"""
import sys
from pathlib import Path
import re
import time

# 添加项目路径（code 目录）
code_dir = Path.cwd().parent if Path.cwd().name == 'utils' else Path.cwd()
sys.path.insert(0, str(code_dir))

from anki.anki import invoke as anki_invoke, ensure_pronunciation_audio
from dictionary.dict import get_word_info_by_word

DECK_NAME = "Media"
SLEEP_TIME = 0.5  # 爬虫间隔，避免请求过快
SKIP_FIRST_N = 0  # 跳过前 n 个笔记

def invoke_with_retry(action: str, retry_times: int = 3, retry_delay: float = 2, **params):
    """带重试机制的 Anki invoke 包装函数"""
    last_error = None
    
    for attempt in range(retry_times):
        try:
            result = anki_invoke(action, **params)
            if result and result.get("error"):
                error_msg = result.get("error", "未知错误")
                if attempt < retry_times - 1:
                    print(f"  ⚠️  Anki 返回错误: {error_msg}, 将在 {retry_delay} 秒后重试...")
                    time.sleep(retry_delay)
                    continue
                else:
                    last_error = error_msg
            else:
                return result
        except Exception as e:
            last_error = str(e)
            if attempt < retry_times - 1:
                print(f"  ⚠️  Anki 连接失败: {e}, 将在 {retry_delay} 秒后重试...")
                time.sleep(retry_delay)
            else:
                raise
    
    if last_error:
        raise Exception(f"Anki 操作失败: {last_error}")
    return result

def has_audio_markup(field_value: str) -> bool:
    """检查字段中是否包含音频标记 [sound:...]"""
    if not field_value:
        return False
    return "[sound:" in field_value

def re_crawl_and_upload_pronunciation(deck_name: str = DECK_NAME, sleep: float = SLEEP_TIME, skip_first_n: int = SKIP_FIRST_N):
    """
    为 Media 牌组中缺少读音的卡片重新爬虫并上传 mp3 发音
    
    Args:
        deck_name: 牌组名称，默认为 "Media"
        sleep: 爬虫请求间隔（秒），避免请求过快
        skip_first_n: 跳过前 n 个笔记，默认为 0（不跳过）
    """
    print(f"开始为牌组 '{deck_name}' 中缺少读音的卡片重新爬虫并上传 mp3...")
    if skip_first_n > 0:
        print(f"⚠️  将跳过前 {skip_first_n} 个笔记")
    print("=" * 60)
    
    # 1. 获取牌组中所有笔记
    query = f'deck:"{deck_name}"'
    note_ids = invoke_with_retry("findNotes", query=query).get("result", [])
    
    if not note_ids:
        print(f"❌ 牌组 '{deck_name}' 中没有找到任何笔记")
        return
    
    print(f"✅ 找到 {len(note_ids)} 个笔记")
    if skip_first_n > 0:
        remaining = len(note_ids) - skip_first_n
        print(f"📊 将处理 {remaining} 个笔记（跳过前 {skip_first_n} 个）")
    print("=" * 60)
    
    # 2. 获取所有笔记的详细信息
    notes_info = invoke_with_retry("notesInfo", notes=note_ids).get("result", [])
    
    success_count = 0
    fail_count = 0
    skip_count = 0
    skipped_by_user = 0
    already_has_audio_count = 0
    
    # 3. 遍历每个笔记
    for i, note_info in enumerate(notes_info, 1):
        # 跳过前 n 个笔记
        if i <= skip_first_n:
            skipped_by_user += 1
            continue
        
        note_id = note_info.get("noteId")
        fields = note_info.get("fields", {})
        word_field = fields.get("Word", {})
        word = word_field.get("value", "").strip() if word_field else ""
        
        if not word:
            print(f"\n[{i}/{len(notes_info)}] ⚠️  跳过：笔记 ID {note_id} 没有 Word 字段")
            skip_count += 1
            continue
        
        print(f"\n[{i}/{len(notes_info)}] 处理单词: {word}")
        
        try:
            # 4. 检查是否已有音频
            pronunciation_field = fields.get("Pronunciation", {})
            pronunciation_value = pronunciation_field.get("value", "") if pronunciation_field else ""
            pos_definitions_field = fields.get("POS_Definitions", {})
            pos_definitions_value = pos_definitions_field.get("value", "") if pos_definitions_field else ""
            
            has_pronunciation_audio = has_audio_markup(pronunciation_value)
            has_pos_audio = has_audio_markup(pos_definitions_value)
            
            if has_pronunciation_audio and has_pos_audio:
                print(f"  ✅ 已有音频，跳过")
                already_has_audio_count += 1
                continue
            
            # 5. 重新爬虫获取单词信息
            print(f"  🔄 正在从 Cambridge Dictionary 获取信息...")
            word_info = get_word_info_by_word(word, sleep=sleep)
            
            if not word_info or not word_info.get("partOfSpeech"):
                print(f"  ⚠️  未获取到单词信息，跳过")
                fail_count += 1
                continue
            
            # 6. 获取发音音频
            audio_markup = ensure_pronunciation_audio(word_info)
            
            if not audio_markup:
                print(f"  ⚠️  未获取到发音音频，跳过")
                fail_count += 1
                continue
            
            print(f"  ✅ 获取到发音音频: {audio_markup}")
            
            # 7. 更新字段
            update_fields = {}
            
            # 更新 Pronunciation 字段
            if not has_pronunciation_audio:
                if pronunciation_value:
                    update_fields["Pronunciation"] = f"{audio_markup}\n{pronunciation_value}"
                else:
                    update_fields["Pronunciation"] = audio_markup
                print(f"  ✅ Pronunciation 字段已更新")
            
            # 更新 POS_Definitions 字段
            if not has_pos_audio:
                if pos_definitions_value:
                    update_fields["POS_Definitions"] = f"{audio_markup}\n{pos_definitions_value}"
                else:
                    update_fields["POS_Definitions"] = audio_markup
                print(f"  ✅ POS_Definitions 字段已更新")
            
            # 8. 更新笔记
            if update_fields:
                try:
                    result = invoke_with_retry("updateNoteFields", note={"id": note_id, "fields": update_fields})
                    
                    if result and not result.get("error"):
                        print(f"  ✅ 笔记更新成功")
                        success_count += 1
                    else:
                        error_msg = result.get("error", "未知错误") if result else "无响应"
                        print(f"  ❌ 更新失败: {error_msg}")
                        fail_count += 1
                except Exception as e:
                    print(f"  ❌ 更新失败（连接问题）: {e}")
                    fail_count += 1
            else:
                print(f"  ⚠️  没有需要更新的字段")
                skip_count += 1
                
        except Exception as e:
            print(f"  ❌ 处理失败: {e}")
            import traceback
            traceback.print_exc()
            fail_count += 1
    
    # 9. 输出统计信息
    print("\n" + "=" * 60)
    print("处理完成！")
    print(f"成功添加音频: {success_count} 个")
    print(f"已有音频: {already_has_audio_count} 个")
    print(f"失败: {fail_count} 个")
    print(f"跳过: {skip_count} 个")
    if skipped_by_user > 0:
        print(f"用户跳过: {skipped_by_user} 个（前 {skip_first_n} 个）")
    print(f"总计: {len(notes_info)} 个")
    print("=" * 60)

# 执行处理
# 修改 SKIP_FIRST_N 的值来跳过前 n 个笔记（例如：SKIP_FIRST_N = 50 表示跳过前 50 个）
if __name__ == "__main__" or True:  # 在 notebook 中总是执行
    re_crawl_and_upload_pronunciation(deck_name=DECK_NAME, sleep=SLEEP_TIME, skip_first_n=SKIP_FIRST_N)


In [ ]:
"""
修复 Media 牌组中失效的 [sound:...] 音频标记
- 查找所有 Media 牌组中的笔记
- 从 Pronunciation 和 POS_Definitions 字段中提取 [sound:...] 标记
- 检查这些音频文件是否真的存在（通过重新上传测试）
- 如果失效，重新爬虫获取单词信息，下载音频并上传
- 更新字段中的音频标记
"""
import sys
from pathlib import Path
import re
import time

# 添加项目路径（code 目录）
code_dir = Path.cwd().parent if Path.cwd().name == 'utils' else Path.cwd()
sys.path.insert(0, str(code_dir))

from anki.anki import invoke as anki_invoke, ensure_pronunciation_audio
from dictionary.dict import get_word_info_by_word

DECK_NAME = "Media"
SLEEP_TIME = 0.5  # 爬虫间隔，避免请求过快
SKIP_FIRST_N = 0  # 跳过前 n 个笔记

def invoke_with_retry(action: str, retry_times: int = 3, retry_delay: float = 2, **params):
    """带重试机制的 Anki invoke 包装函数"""
    last_error = None
    
    for attempt in range(retry_times):
        try:
            result = anki_invoke(action, **params)
            if result and result.get("error"):
                error_msg = result.get("error", "未知错误")
                if attempt < retry_times - 1:
                    print(f"  ⚠️  Anki 返回错误: {error_msg}, 将在 {retry_delay} 秒后重试...")
                    time.sleep(retry_delay)
                    continue
                else:
                    last_error = error_msg
            else:
                return result
        except Exception as e:
            last_error = str(e)
            if attempt < retry_times - 1:
                print(f"  ⚠️  Anki 连接失败: {e}, 将在 {retry_delay} 秒后重试...")
                time.sleep(retry_delay)
            else:
                raise
    
    if last_error:
        raise Exception(f"Anki 操作失败: {last_error}")
    return result

def extract_sound_markups(field_value: str) -> list:
    """从字段中提取所有 [sound:...] 标记"""
    if not field_value:
        return []
    
    pattern = r'\[sound:([^\]]+)\]'
    matches = re.findall(pattern, field_value)
    return matches

def fix_invalid_audio_markups(deck_name: str = DECK_NAME, sleep: float = SLEEP_TIME, skip_first_n: int = SKIP_FIRST_N):
    """
    修复 Media 牌组中失效的 [sound:...] 音频标记
    
    Args:
        deck_name: 牌组名称，默认为 "Media"
        sleep: 爬虫请求间隔（秒），避免请求过快
        skip_first_n: 跳过前 n 个笔记，默认为 0（不跳过）
    """
    print(f"开始修复牌组 '{deck_name}' 中失效的音频标记...")
    if skip_first_n > 0:
        print(f"⚠️  将跳过前 {skip_first_n} 个笔记")
    print("=" * 60)
    
    # 1. 获取牌组中所有笔记
    query = f'deck:"{deck_name}"'
    note_ids = invoke_with_retry("findNotes", query=query).get("result", [])
    
    if not note_ids:
        print(f"❌ 牌组 '{deck_name}' 中没有找到任何笔记")
        return
    
    print(f"✅ 找到 {len(note_ids)} 个笔记")
    if skip_first_n > 0:
        remaining = len(note_ids) - skip_first_n
        print(f"📊 将处理 {remaining} 个笔记（跳过前 {skip_first_n} 个）")
    print("=" * 60)
    
    # 2. 获取所有笔记的详细信息
    notes_info = invoke_with_retry("notesInfo", notes=note_ids).get("result", [])
    
    success_count = 0
    fail_count = 0
    skip_count = 0
    skipped_by_user = 0
    no_audio_markup_count = 0
    fixed_count = 0
    
    # 3. 遍历每个笔记
    for i, note_info in enumerate(notes_info, 1):
        # 跳过前 n 个笔记
        if i <= skip_first_n:
            skipped_by_user += 1
            continue
        
        note_id = note_info.get("noteId")
        fields = note_info.get("fields", {})
        word_field = fields.get("Word", {})
        word = word_field.get("value", "").strip() if word_field else ""
        
        if not word:
            print(f"\n[{i}/{len(notes_info)}] ⚠️  跳过：笔记 ID {note_id} 没有 Word 字段")
            skip_count += 1
            continue
        
        print(f"\n[{i}/{len(notes_info)}] 处理单词: {word}")
        
        try:
            # 4. 提取所有音频标记
            pronunciation_field = fields.get("Pronunciation", {})
            pronunciation_value = pronunciation_field.get("value", "") if pronunciation_field else ""
            pos_definitions_field = fields.get("POS_Definitions", {})
            pos_definitions_value = pos_definitions_field.get("value", "") if pos_definitions_field else ""
            
            pronunciation_sounds = extract_sound_markups(pronunciation_value)
            pos_sounds = extract_sound_markups(pos_definitions_value)
            all_sounds = list(set(pronunciation_sounds + pos_sounds))
            
            if not all_sounds:
                print(f"  ⚠️  未找到音频标记，跳过")
                no_audio_markup_count += 1
                skip_count += 1
                continue
            
            print(f"  📋 找到 {len(all_sounds)} 个音频标记: {', '.join(all_sounds)}")
            
            # 5. 重新爬虫获取单词信息并下载音频
            print(f"  🔄 正在从 Cambridge Dictionary 重新获取音频...")
            word_info = get_word_info_by_word(word, sleep=sleep)
            
            if not word_info or not word_info.get("partOfSpeech"):
                print(f"  ⚠️  未获取到单词信息，跳过")
                fail_count += 1
                continue
            
            # 6. 获取新的发音音频
            new_audio_markup = ensure_pronunciation_audio(word_info)
            
            if not new_audio_markup:
                print(f"  ⚠️  未获取到发音音频，跳过")
                fail_count += 1
                continue
            
            print(f"  ✅ 获取到新的发音音频: {new_audio_markup}")
            
            # 7. 检查是否需要更新字段
            update_fields = {}
            
            # 检查 Pronunciation 字段
            if pronunciation_sounds:
                # 如果字段中有音频标记，但可能失效，用新的替换
                # 替换所有旧的音频标记为新的
                new_pronunciation = pronunciation_value
                pronunciation_updated = False
                for old_sound in pronunciation_sounds:
                    old_markup = f"[sound:{old_sound}]"
                    if old_markup in new_pronunciation:
                        new_pronunciation = new_pronunciation.replace(old_markup, new_audio_markup, 1)
                        pronunciation_updated = True
                
                if pronunciation_updated:
                    update_fields["Pronunciation"] = new_pronunciation
                    print(f"  ✅ Pronunciation 字段将更新（替换失效的音频标记）")
            elif not has_audio_markup(pronunciation_value):
                # 如果字段中没有音频标记，添加新的
                if pronunciation_value:
                    update_fields["Pronunciation"] = f"{new_audio_markup}\n{pronunciation_value}"
                else:
                    update_fields["Pronunciation"] = new_audio_markup
                print(f"  ✅ Pronunciation 字段将添加音频")
            
            # 检查 POS_Definitions 字段
            if pos_sounds:
                # 如果字段中有音频标记，但可能失效，用新的替换
                new_pos_definitions = pos_definitions_value
                pos_updated = False
                for old_sound in pos_sounds:
                    old_markup = f"[sound:{old_sound}]"
                    if old_markup in new_pos_definitions:
                        new_pos_definitions = new_pos_definitions.replace(old_markup, new_audio_markup, 1)
                        pos_updated = True
                
                if pos_updated:
                    update_fields["POS_Definitions"] = new_pos_definitions
                    print(f"  ✅ POS_Definitions 字段将更新（替换失效的音频标记）")
            elif not has_audio_markup(pos_definitions_value):
                # 如果字段中没有音频标记，添加新的
                if pos_definitions_value:
                    update_fields["POS_Definitions"] = f"{new_audio_markup}\n{pos_definitions_value}"
                else:
                    update_fields["POS_Definitions"] = new_audio_markup
                print(f"  ✅ POS_Definitions 字段将添加音频")
            
            # 8. 更新笔记
            if update_fields:
                try:
                    result = invoke_with_retry("updateNoteFields", note={"id": note_id, "fields": update_fields})
                    
                    if result and not result.get("error"):
                        print(f"  ✅ 笔记更新成功")
                        success_count += 1
                        fixed_count += 1
                    else:
                        error_msg = result.get("error", "未知错误") if result else "无响应"
                        print(f"  ❌ 更新失败: {error_msg}")
                        fail_count += 1
                except Exception as e:
                    print(f"  ❌ 更新失败（连接问题）: {e}")
                    fail_count += 1
            else:
                print(f"  ⚠️  音频标记正常，无需更新")
                skip_count += 1
                
        except Exception as e:
            print(f"  ❌ 处理失败: {e}")
            import traceback
            traceback.print_exc()
            fail_count += 1
    
    # 9. 输出统计信息
    print("\n" + "=" * 60)
    print("处理完成！")
    print(f"成功修复: {fixed_count} 个")
    print(f"成功添加: {success_count - fixed_count} 个")
    print(f"失败: {fail_count} 个")
    print(f"跳过: {skip_count} 个")
    print(f"无音频标记: {no_audio_markup_count} 个")
    if skipped_by_user > 0:
        print(f"用户跳过: {skipped_by_user} 个（前 {skip_first_n} 个）")
    print(f"总计: {len(notes_info)} 个")
    print("=" * 60)

def has_audio_markup(field_value: str) -> bool:
    """检查字段中是否包含音频标记 [sound:...]"""
    if not field_value:
        return False
    return "[sound:" in field_value

# 执行处理
# 修改 SKIP_FIRST_N 的值来跳过前 n 个笔记（例如：SKIP_FIRST_N = 50 表示跳过前 50 个）
if __name__ == "__main__" or True:  # 在 notebook 中总是执行
    fix_invalid_audio_markups(deck_name=DECK_NAME, sleep=SLEEP_TIME, skip_first_n=SKIP_FIRST_N)


开始修复牌组 'Media' 中失效的音频标记...
✅ 找到 399 个笔记

[1/399] 处理单词: twilight
  📋 找到 1 个音频标记: twilight-us-2aa6763e.mp3
  🔄 正在从 Cambridge Dictionary 重新获取音频...
  ✅ 获取到新的发音音频: [sound:twilight-us-2aa6763e.mp3]
  ✅ Pronunciation 字段将更新（替换失效的音频标记）
  ✅ POS_Definitions 字段将添加音频
  ✅ 笔记更新成功

[2/399] 处理单词: encapsulation
  📋 找到 1 个音频标记: encapsulation-us-68621333.mp3
  🔄 正在从 Cambridge Dictionary 重新获取音频...
  ✅ 获取到新的发音音频: [sound:encapsulation-us-68621333.mp3]
  ✅ Pronunciation 字段将更新（替换失效的音频标记）
  ✅ POS_Definitions 字段将添加音频
  ✅ 笔记更新成功

[3/399] 处理单词: transcend
  📋 找到 1 个音频标记: transcend-us-77c9b961.mp3
  🔄 正在从 Cambridge Dictionary 重新获取音频...
  ✅ 获取到新的发音音频: [sound:transcend-us-77c9b961.mp3]
  ✅ Pronunciation 字段将更新（替换失效的音频标记）
  ✅ POS_Definitions 字段将添加音频
  ✅ 笔记更新成功

[4/399] 处理单词: vest
  📋 找到 1 个音频标记: vest-us-a0f3b9f4.mp3
  🔄 正在从 Cambridge Dictionary 重新获取音频...
  ✅ 获取到新的发音音频: [sound:vest-us-a0f3b9f4.mp3]
  ✅ Pronunciation 字段将更新（替换失效的音频标记）
  ✅ POS_Definitions 字段将添加音频
  ✅ 笔记更新成功

[5/399] 处理单词: clipboard
  📋 找到 1 个音频标记: clipboard